# Building an Email Spam Filter

To classify messages as spam or non-spam, the computer:

- Learns how humans classify messages
- Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam
- Classifies a new message based on these probability values 
    - If the probability for spam is greater, then it classifies the message as spam
    - Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message)

So my first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

### The Goal
**My goal is to create a spam filter that classifies new messages with an accuracy greater than 80%.**

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/).

## Exploring the Data

### Opening the Dataset

I'll use some simple Python and pandas operations to explore the dataset.

In [1]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', 
                       sep = '\t', 
                       header = None, 
                       names = ['Label', 'SMS']
                      )

In [2]:
print(sms_spam.shape)
print('\n')
print(sms_spam.head())

(5572, 2)


  Label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


### Relative Frequencies of Spam and Non-Spam Messages

It appears there are two columns of interest: 'Label' and 'SMS'.

I'll generate a quick relative frequency table for the values in the 'Label' column to identify how many of the messages in the provided dataset are either spam or ham messages.

In [3]:
print(sms_spam['Label'].value_counts(normalize = True))

ham     0.865937
spam    0.134063
Name: Label, dtype: float64


Above, we see that about 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative of emails in general, since in practice most messages that people receive are ham.

## Testing the Filter Pre-Creation

### Reasoning 
About 87% of the messages are ham ("ham" means non-spam), and the remaining 13% are spam. I can move on to building the spam filter.

However, before creating it, it's very helpful to first think of a way of testing how well it works. When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once the spam filter is done, I'll need to test how good it is with classifying new messages. To test the spam filter, I'm going to split our dataset into two categories:

- A training set, which we'll use to "train" the computer how to classify messages.
- A test set, which we'll use to test how good the spam filter is with classifying new messages.

I'll keep 80% of our dataset for training, and 20% for testing (to train the algorithm on as much data as possible, but to also have enough test data). The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

*To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in the test set are already classified by a human. When the spam filter is ready, I'll treat these messages as new and have the filter classify them. Once I have the results, I'll be able to compare the algorithm classification with that done by a human, and this way I'll see how good the spam filter really is.*

### Randomizing My Dataset

In [4]:
# randomize the data set
randomized_data = sms_spam.sample(frac = 1, random_state = 1)

# create an index for splitting the data set, rounded to avoid decimals
splitting_index = round(len(randomized_data) * 0.8) # =4458

# using the index to split the data set into training set and testing set
training_set = randomized_data[:splitting_index].reset_index(drop = True)
testing_set = randomized_data[splitting_index:].reset_index(drop = True)

After creating the training and testing sets, it's important to check whether or not the sets accurately represent the distribution of ham and spam messages as the original data set.

In [5]:
training_set['Label'].value_counts(normalize = True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [6]:
testing_set['Label'].value_counts(normalize = True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

Above, we see that both the training and testing sets accurately represent the distribution of ham and spam messages as the original set.

## Cleaning the Data

After splitting the dataset into a training set and a test set, the next big step is to use the training set to teach the algorithm to classify new messages. To calculate all probabilities, I'll first need to perform a bit of data cleaning. I'll start by removing the punctuation and bringing all the words to lower case.

This is what the dataset looks like before cleaning:

In [7]:
print(training_set.head())

  Label                                                SMS
0   ham                       Yep, by the pretty sculpture
1   ham      Yes, princess. Are you going to make me moan?
2   ham                         Welp apparently he retired
3   ham                                            Havent.
4   ham  I forgot 2 ask ü all smth.. There's a card on ...


And this is what our data set looks like after cleaning using some simple string operations:

In [8]:
# strip any character not from a-z, A-Z, or 0-9
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')

# transform every letter in every word To lower case
training_set['SMS'] = training_set['SMS'].str.lower()

print(training_set.head())

  Label                                                SMS
0   ham                       yep  by the pretty sculpture
1   ham      yes  princess  are you going to make me moan 
2   ham                         welp apparently he retired
3   ham                                            havent 
4   ham  i forgot 2 ask ü all smth   there s a card on ...


## The Approach to the Problem

Everyone gets spam messages and knows how annoying it can be. There are particular defining qualities of spam messages that make them so distinct, and the content of the messages always seems to be similar in nature. 

I will create a list with all of the unique words that occur in the messages of our training set. The vocabulary should be compiled into a Python list containing all the unique words across all messages, where each word is represented as a string.

In [9]:
# transforming each message into a list split by space characters
training_set['SMS'] = training_set['SMS'].str.split()

# initiating an empty list
vocabulary = []

for each_list in training_set['SMS']:
    for each_word in each_list:
        vocabulary.append(each_word)

vocabulary = set(vocabulary) # to get rid of duplicates
vocabulary = list(vocabulary)

In [10]:
len(vocabulary)

7783

It looks like there are 7,783 unique words in all the messages of our training set.

# Transforming the Dataset

After creating the vocabulary for the messages in the training set, I'm going to use the vocabulary to transform the data. I want to create a dictionary that shows how many times each particular word in the vocabulary dictionary is being used. 

Eventually, I'm going to create a new DataFrame. However, I'll first build a dictionary named 'word_counts_per_sms' that I'll then convert.

### Creating the New Dictionary

In [11]:
# Each key is a unique word (a string) from the vocabulary
# Each value is a list of the length of training set, where each element in the list is a 0
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

# loop over training_set['SMS'] using the enumerate() function to get both the index and the SMS message (index and sms)
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

### Transforming the Dictionary into a Pandas DataFrame

To visualize what the code above has done, I'm converting the new dictionary into a DataFrame. This new DataFrame shows the count of each respective word found in the message.

In [12]:
# Transform word_counts_per_sms into a DataFrame using pd.DataFrame()
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,bar,doesnt,accordingly,fifty,8,profit,prakasam,rcvd,isnt,bhaskar,...,ure,broke,pap,tiz,roger,worlds,rayman,oops,buttons,accidentally
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Putting It All Together

I'll concatenate the DataFrame I just built above with the DataFrame containing the training set. This way, I'll also have the Label and the SMS columns.

In [13]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,bar,doesnt,accordingly,fifty,8,profit,prakasam,rcvd,...,ure,broke,pap,tiz,roger,worlds,rayman,oops,buttons,accidentally
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


As you can see above, I now have a DataFrame with Labels, the content of the messages themselves, and the count of each respective word found in the message.

# Building the Filter with Naive Bayes

I'm now done with cleaning the training set and can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

$$
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$$$
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
$$
Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$
Some of the terms in the four equations above will have the same value for every new message. I can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, I'll use the training set to calculate:

P(Spam) and P(Ham)
NSpam, NHam, NVocabulary

I'll also use Laplace smoothing and set $\alpha = 1$.

In [14]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# Calculating N_spam, N_ham, N_vocabulary
count_of_spam_words_per_sms = spam_messages['SMS'].apply(len)
n_spam = count_of_spam_words_per_sms.sum()

count_of_ham_words_per_sms = ham_messages['SMS'].apply(len)
n_ham = count_of_ham_words_per_sms.sum()

n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

### Calculating Parameters

Now that I have the constant terms calculated above, I can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

In [15]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

Below you can see an example of how the dictionaries created now hold all parameters for words in the vocabulary.

In [16]:
print(parameters_spam['hey'])
print(parameters_ham['hey'])
print(parameters_spam['free'])
print(parameters_ham['free'])

0.00021764680276846734
0.0014303291294986158
0.00766116745745005
0.0007536142725315288


# Applying Created Parameters to Build the Filter

Now that I have all our parameters calculated, I can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn).
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

### Defining a Function for Spam Classification

In [17]:
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

### The Filter Demonstrated

Below, we demonstrate the created spam filter in action:

In [18]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [19]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Using the Filter on the Test Set

The two results above look promising, but let's see how well the filter does on the test set, which has 1,114 messages.

I'll start by writing a function that returns classification labels instead of printing them.

In [20]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that I have a function that returns labels instead of printing them, I can use it to create a new column in the test set.

In [21]:
testing_set['predicted'] = testing_set['SMS'].apply(classify_test_set)
testing_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


## Applying the Filter on the Test Set

The filter is created and applied to the test set. This data has been labeled but was not used in the training of the model.

In [22]:
# measure the accuracy of the spam filter

correct = 0
total = len(testing_set)

for each_row in testing_set.iterrows():
    row_data = each_row[1]
    if row_data['Label'] == row_data['predicted']:
        correct += 1

print('Correct Categorization:', correct)
print('Incorrect Categorization:', total - correct)
print('Filter Accuracy:', round(correct/total * 100, 2), '%')

Correct Categorization: 1100
Incorrect Categorization: 14
Filter Accuracy: 98.74 %


The accuracy is close to 98.74%, which is really good. The spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.

### Next Steps
In this project, I managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set I used, which is a pretty good result. The initial goal was an accuracy of over 90%, and I managed to do much better than that.

Future steps include:

- Analyze the 14 messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly
- Make the filtering process more complex by making the algorithm sensitive to letter case